In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import json
import pandas as pd

with open('/kaggle/input/flicktime/movielens1.json', 'r') as f:
    data = json.load(f)

tmdb_data = pd.json_normalize(data)

In [ ]:
tmdb_data.shape

In [ ]:
# genome_scores = pd.read_csv('/kaggle/input/flicktime/genome_scores.csv')
# genome_tags = pd.read_csv('/kaggle/input/flicktime/genome_tags.csv')
link = pd.read_csv('/kaggle/input/flicktime/link.csv')
# movie = pd.read_csv('/kaggle/input/flicktime/movie.csv')
rating = pd.read_csv('/kaggle/input/flicktime/rating.csv')
# tag = pd.read_csv('/kaggle/input/flicktime/tag.csv')

In [ ]:
rating.head()

In [ ]:
tmdb_data.isna().sum()/tmdb_data.shape[0] * 100

In [ ]:
tmdb_data.drop(['belongs_to_collection', 'belongs_to_collection.backdrop_path', 'belongs_to_collection.poster_path',
               'belongs_to_collection.name','belongs_to_collection.id'],inplace = True, axis = 1)

In [ ]:
tmdb_data.dropna(inplace = True)

In [ ]:
tmdb_data.isna().sum()

In [ ]:
merged_df = tmdb_data.merge(link,left_on='id', right_on = 'tmdbId')

In [ ]:
import gc
del tmdb_data
gc.collect()

In [ ]:
movie_ids = merged_df.movieId.values

In [ ]:
len(movie_ids)

In [ ]:
merged_rating = rating.loc[rating['movieId'].isin(movie_ids)]

In [ ]:
import gc
del rating
del merged_df
gc.collect()

In [ ]:
merged_rating.head()

In [ ]:
merged_rating.shape

In [ ]:
del link
gc.collect()

In [ ]:
from sklearn.model_selection import train_test_split

train_ratings, test_ratings = train_test_split(merged_rating, test_size=0.2, random_state=42)

In [ ]:
!pip install surprise
import surprise
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

In [ ]:
# Define the reader and load the dataset
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(train_ratings[['userId', 'movieId', 'rating']], reader)

In [ ]:
import gc
del merged_rating
gc.collect()

In [ ]:
%%time
# Define the SVD model and cross-validate
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# Fit the SVD model to the entire training set
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
import pickle

In [ ]:
file_name = 'svd_model.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(svd, f)

In [ ]:
surprise.dump.dump('svd', predictions=None, algo=svd)

In [ ]:
# data.to_pickle("./dummy.pkl")  

In [ ]:
import gc
del data

In [ ]:
gc.collect()

In [ ]:
from surprise import accuracy

# Convert the test set to Surprise format
testset = list(zip(test_ratings['userId'], test_ratings['movieId'], test_ratings['rating']))

In [ ]:
# Make predictions and compute accuracy metrics
predictions = svd.test(testset)
accuracy.rmse(predictions)
accuracy.mae(predictions)

In [ ]:
predictions

In [ ]:
!surprise -h

In [ ]:
surprise.dump.dump('svd', predictions=None, algo=svd)